In [ ]:
# -*- coding: utf-8 -*-

"""Подключаем зависимости"""

from IPython.display import display
import wfdb
from wfdb import processing
import pylab
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
from scipy import sparse
from scipy.sparse import linalg
from scipy.signal import chirp, find_peaks, peak_widths
from numpy.linalg import norm
from scipy.interpolate import CubicSpline
from scipy.interpolate import splrep, splev
import pandas as pd
import os
from pathlib import Path

"""Получаем пути папок"""

from pathlib import Path
path_of_the_directory = 'C:\\Users\\dnbne\\Desktop\\ECG\\data'
file = Path(path_of_the_directory ).glob('*')
folder_paths=[]
for i in file:
    folder_paths.append(i)

folder_paths.sort()
len_folder_paths=len(folder_paths)

i=0

"""Создаем датафрейм"""

column_names = ['pat_id', 'cardio_id', 'heartbeat_id', 'age', 'sex', 'n_peaks', 'len_peaks', 'len_avg', 'average_y', 'y_mean']
data=pd.DataFrame(columns = column_names)

"""Цикл по папкам i, файлам j и сердцебиениям n"""

for i in range (len_folder_paths):
    os.chdir(folder_paths[i])
    file_paths=[]
    path_of_the_directory = folder_paths[i]
    file = Path(path_of_the_directory ).glob('*')

    """Получаем имена файлов в папке"""

    for o in file:
        file_name=Path(o).stem
        file_paths.append(file_name)

    file_paths=list(set(file_paths))
    file_paths.sort()
    len_file_paths=len(file_paths)
    # print(len_file_paths)
   
    for j in range (len_file_paths):

        """1. Препроцессинг данных"""

        #Читаем файлы с ЭКГ
        try:
            record = wfdb.rdsamp(file_paths[j])
        except:pass
        #Так как record - это tuple, где первый элемент - массив 10000*2, то вытаскиваем этот массив
        #Он и является данными по оси y
        #Этот массив содержит данные для фильтрованного и нефильтрованной ЭКГ
        record_array=record[0]
        #Создаем новый список, в который будем добавлять значения по оси y фильтрованной ЭКГ
        filt_list=[]

        #Наполняем список
        for s in range(len(record_array)):
            filt_list.append(record_array[s][1])

        #Создаем список, который содержит значения по оси x
        x=list(range(0,10000))

        #Отрисовка аналоговой ЭКГ
        # an_record=wfdb.rdrecord('rec_1')
        # wfdb.plot_wfdb(record=an_record, figsize=(20,16))

        #Отрисовка цифровой ЭКГ
        # figure(figsize=(25, 8))
        # plt.plot(x,filt_list)
        # plt.xlim([0, 1200])
        # plt.show()

        """Выравнивание графика по 0"""

        def baseline_arPLS(y, ratio=1e-6, lam=100, niter=10, full_output=False):
            L = len(y)
            diag = np.ones(L - 2)
            D = sparse.spdiags([diag, -2*diag, diag], [0, -1, -2], L, L - 2)
            H = lam * D.dot(D.T) 
            w = np.ones(L)
            W = sparse.spdiags(w, 0, L, L)
            crit = 1
            count = 0
            while crit > ratio:
                z = linalg.spsolve(W + H, W * y)
                d = y - z
                dn = d[d < 0]
                m = np.mean(dn)
                s = np.std(dn)
                w_new = 1 / (1 + np.exp(2 * (d - (2*s - m))/s))
                crit = norm(w_new - w) / norm(w)
                w = w_new
                W.setdiag(w)  
                count += 1
                if count > niter:
                    break
            if full_output:
                info = {'num_iter': count, 'stop_criterion': crit}
                return z, d, info
            else:
                return z

        poly = CubicSpline(x, filt_list)
        baseline = poly(x)

        ano_filt_list = filt_list + baseline 

        _, mod_filt_list, info = baseline_arPLS(ano_filt_list, lam=1e6, niter=100,
                                                full_output=True)

        # figure(figsize=(25, 8))
        # plt.plot(x,mod_filt_list)
        # plt.xlim([0, 1200])

        """Сглаживаем график"""

        bspl = splrep(x,mod_filt_list,s=5)
        bspl_y = splev(x,bspl)
        # plt.figure()
        # figure(figsize=(25, 8))
        # plt.plot(x,bspl_y)
        # plt.xticks(np.arange(0,10000,25))
        # plt.xlim([0, 1200])

        # print (bspl_y[0])

        """2. Извлечение признаков. Код ниже позволяет найти и пики, и провалы на графике"""

        all_peaks=processing.find_peaks(bspl_y)
        hard_peaks=all_peaks[0]

        """Код ниже позволяет найти только пики"""

        def find_feature(ecg, k):
            if (k == 1) :
                peaks_array.append(arr[0])
            if (arr[0] >= arr[1]) :
                peaks_array.append(arr[0])
            if (arr[k - 1] >= arr[k - 2]) :
                peaks_array.append(arr[k - 1])

            for l in range(1, k - 1) :
                if (arr[l] >= arr[l - 1] and arr[l] >= arr[l + 1]) :
                    peaks_array.append(l)
                    
        arr = bspl_y
        k = len(arr)
        peaks_array=[]
        find_feature(bspl_y, k)

        """Блок ниже рассчитывает ширину всех пиков"""

        # results_full = peak_widths(bspl_y, peaks_array, rel_height=0.6)
        # np.set_printoptions(precision=3, suppress=True)
        # results_full[0]

        """Создаем словарь, где ключ - точки по оси х, значение - по оси у"""

        hard_peaks = dict(zip(x, bspl_y))

        """Создаем список, который содержит все точки по оси х, которые имеют значение больше 1 (так как именно эти значения соответсвуют максимальному пику в сердцебиении - пику R)"""

        high_peaks=dict()
        for key, value in hard_peaks.items():
            if value > 0.75:
                high_peaks[key] = value
            else:
                high_peaks[key] = 0
            

        """Но нам нужны не все точки по оси х, а только та, что имеет максимум по оси у"""

        max_peaks = []
        for key, value in high_peaks.items():
            if key != 9999 and key != 0:
                if high_peaks[key] > high_peaks[key + 1] and high_peaks[key] > high_peaks[key - 1]:
                    max_peaks.append(key)
        
        len_heartbeat=len(max_peaks)

        hea_array=record[1]

        age=hea_array['comments'][0][5:8]

        """Так как пол в 'comments' - строка, то преобразуем в число. 0 - мужчина, 1 - женщина"""

        hea_array['comments'][1][5:11]

        sex=0
        if hea_array['comments'][1][5:11] == 'male': sex=0
        else: sex=1

        high_peaks=dict()
        for key, value in hard_peaks.items():
            if abs(value) > 0.75:
                high_peaks[key] = value
            else:
                high_peaks[key] = 0

        max_peaks = []
        for key, value in high_peaks.items():
            if key != 9999 and key != 0:
                if abs(high_peaks[key]) > abs(high_peaks[key + 1]) and abs(high_peaks[key]) > abs(high_peaks[key - 1]):
                    max_peaks.append(key)

        peaks_array = peaks_array[1:]

        peaks_per_beat = []
        for ind in range(len(max_peaks)):
            peaks_per_beat.append([])

        ind = 0
    
        for item in peaks_array:
            if ind >= len(max_peaks):
                break
            elif item <= max_peaks[ind]:
                peaks_per_beat[ind].append(item)
            else:
                ind += 1

        ind = 0
        jnd = 0
        peaks_count = []  # кол-во пиков в сердцебиении

        peaks_distance = []  # длины между соседними пиками
        for ind in range(len(max_peaks)):
            peaks_distance.append([])
    
        mean_dist_between_peaks = []  # среднее расстояние между пиками в одном сердцебиении

        for ind in range(len(peaks_per_beat)):
            peaks_count.append(len(peaks_per_beat[ind]))
            for jnd in range(1, len(peaks_per_beat[ind])):
                    peaks_distance[ind].append(peaks_per_beat[ind][jnd] - peaks_per_beat[ind][jnd - 1])
            mean_dist_between_peaks.append(np.mean(peaks_distance[ind]))
    
        mean_y_value = []  # среднее занчение по У в одном сердцебиении

        sum = 0

        for ind in range(len(peaks_per_beat)):
            peaks_count.append(len(peaks_per_beat[ind]))
            try:
                for jnd in range(len(peaks_per_beat[ind])):
                    sum += hard_peaks[peaks_per_beat[ind][jnd]]
                    mean_y_value.append(sum/len(peaks_per_beat[ind]))
            except KeyError:pass
            except ZeroDivisionError:pass
            
        sum = 0

        y_mean=np.mean(bspl_y)

        """Наполняем датафрейм фичами, проходимся по сердцебиениям"""
        for n in range(len_heartbeat):
            try:
                col_data = {'pat_id': i+1, 'cardio_id': j+1, 'heartbeat_id': n+1, 'age': age, 'sex': sex, 
                'n_peaks':peaks_distance[n][0], 'len_peaks':peaks_distance[n][0], 
                'len_avg':mean_dist_between_peaks[n], 'average_y':mean_y_value[n], 'y_mean':y_mean}
                data = data.append(col_data, ignore_index = True)
            except IndexError:pass

In [ ]:
print(data)

In [115]:
#Сохраняем итоговый датафрейм для дальнейшего использования в обучении
data.to_csv(r'C:\Users\dnbne\Desktop\big_patient_data.csv', sep=',', encoding='utf-8')